In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

%matplotlib inline

np.random.seed(1)
srcs = "/home/aa940249tw/nctu/ml_chip/Lab_2/lab2-1/srcs/"

In [2]:
torch.cuda.is_available()

True

In [3]:
def load_param(conv_num, fc_num):
    parameters = {}
    for i in range(1, conv_num + 1):
        parameters['W' + str(i)] = np.load(srcs + "Conv" + str(i) + "_weights.npy")
        parameters['b' + str(i)] = np.load(srcs + "Conv" + str(i) + "_bias.npy")
    
    for i in range(conv_num + 1, conv_num + 1 + fc_num):
        parameters['W' + str(i)] = np.load(srcs + "Fc" + str(i) + "_weights.npy")
        parameters['b' + str(i)] = np.load(srcs + "Fc" + str(i) + "_bias.npy")
    return parameters


In [4]:
class vgg16:
    def __init__(self, params):
        self.params = params

    def relu(self, Z):
        A = np.maximum(0, Z)
        return A
    
    def zero_padding(self, X, pad):
        X_pad = np.pad(X, ((0, 0), (0, 0), (pad, pad), (pad, pad)), 'constant', constant_values=0)
        return X_pad
    
    def linear_foward(self, A_prev, W, b):
        b = np.reshape(b, (b.shape[0], 1))
        Z = np.dot(W, A_prev) + b
        print(Z.shape)
        assert (Z.shape == (W.shape[0], A_prev.shape[1]))
        return Z
    
    def max_pooling(self, A_prev, hparameters = {'f':2, 'stride':2}):
        (m, n_C_prev, n_H_prev, n_W_prev) = A_prev.shape
        f = hparameters["f"]
        stride = hparameters["stride"]
        n_H = int(1 + (n_H_prev - f) / stride)
        n_W = int(1 + (n_W_prev - f) / stride)
        n_C = n_C_prev

        A = np.zeros((m, n_C, n_H, n_W)) 
        for i in range(m):
            a_prev = A_prev[i]
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        v_start = h * stride
                        v_end = v_start + f
                        h_start =  w * stride
                        h_end = h_start + f

                        a_prev_slice = a_prev[c, v_start:v_end, h_start:h_end]
                        A[i, c, h, w] = np.max(a_prev_slice)
        return A

    def conv_single(self, a_slice_prev, w, b):
        s = np.multiply(a_slice_prev, w)
        Z = np.sum(s) + b
        return self.relu(Z)
    
    def conv_foward(self, A_prev, W, b, hparameters = {'stride':1, 'pad':1}):
        (m, n_C_prev, n_H_prev, n_W_prev) = A_prev.shape
        (n_C, n_C_prev, f, f) = W.shape
        stride = hparameters['stride']
        pad = hparameters['pad']
        n_H = int((n_H_prev - f + 2 * pad) / stride) + 1
        n_W = int((n_W_prev - f + 2 * pad) / stride) + 1
        Z = np.zeros((m, n_C, n_H, n_W))
        A_prev_pad = self.zero_padding(A_prev, pad)

        for i in range(m):
            a_prev_pad = A_prev_pad[i]
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        v_start = h * stride
                        v_end = v_start + f
                        h_start =  w * stride
                        h_end = h_start + f
                        a_slice_prev = a_prev_pad[:, v_start:v_end, h_start:h_end]
                        Z[i, c, h, w] = self.conv_single(a_slice_prev, W[c, ...], b[c, ...])
    
        assert(Z.shape == (m, n_C, n_H, n_W))
        return Z

    def foward(self, x):
        print('Conv1...')
        out = self.conv_foward(x, self.params['W1'], self.params['b1'])
        print('Conv2...')
        out = self.conv_foward(out, self.params['W2'], self.params['b2'])
        out = self.max_pooling(out)
        print('Conv3...')
        out = self.conv_foward(out, self.params['W3'], self.params['b3'])
        print('Conv4...')
        out = self.conv_foward(out, self.params['W4'], self.params['b4'])
        out = self.max_pooling(out)
        print('Conv5...')
        out = self.conv_foward(out, self.params['W5'], self.params['b5'])
        print('Conv6...')
        out = self.conv_foward(out, self.params['W6'], self.params['b6'])
        print('Conv7...')
        out = self.conv_foward(out, self.params['W7'], self.params['b7'])
        out = self.max_pooling(out)
        print('Conv8...')
        out = self.conv_foward(out, self.params['W8'], self.params['b8'])
        print('Conv9...')
        out = self.conv_foward(out, self.params['W9'], self.params['b9'])
        print('Conv10...')
        out = self.conv_foward(out, self.params['W10'], self.params['b10'])
        out = self.max_pooling(out)
        print('Conv11...')
        out = self.conv_foward(out, self.params['W11'], self.params['b11'])
        print('Conv12...')
        out = self.conv_foward(out, self.params['W12'], self.params['b12'])
        print('Conv13...')
        out = self.conv_foward(out, self.params['W13'], self.params['b13'])
        out = self.max_pooling(out)
        print(out)
        out = torch.from_numpy(out)
        out = F.adaptive_avg_pool2d(out, [7, 7]).detach().cpu()
        out = out.reshape(-1, out.size(0)).numpy()
        print(out.shape)
        print('FC1...')
        out = self.linear_foward(out, self.params['W14'], self.params['b14'])
        out = self.relu(out)
        print('FC2...')
        out = self.linear_foward(out, self.params['W15'], self.params['b15'])
        out = self.relu(out)
        print('FC3...')
        out = self.linear_foward(out, self.params['W16'], self.params['b16'])
        return out

In [5]:
def VGG16():
    params = load_param(13, 3)
    params['W1'].shape
    return vgg16(params)

In [6]:
params = load_param(13, 3)
params['b1'].shape

(64,)

In [7]:
img_to_tensor = transforms.ToTensor()
model = VGG16()
img = Image.open('srcs/input.jpg')
img = img.resize((224, 224))
tensor = img_to_tensor(img)
tensor=tensor.resize_(1,3,224,224)
input = tensor.detach().cpu().numpy()
ans = model.foward(input)

Conv1...
Conv2...
Conv3...
Conv4...
Conv5...
Conv6...
Conv7...
Conv8...
Conv9...
Conv10...
Conv11...
Conv12...
Conv13...
[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 8.00150405e-01 ... 0.00000000e+00
    2.64070100e-01 0.00000000e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+

In [8]:
ans

array([[-1.83044914e+00],
       [-5.41466700e-01],
       [-3.00663905e-01],
       [ 1.31901698e+00],
       [ 6.21482767e-01],
       [ 1.72802377e+00],
       [ 3.11241977e-01],
       [-7.72665661e-01],
       [-2.33660673e+00],
       [-4.15072461e+00],
       [-3.64292849e+00],
       [-5.22179888e+00],
       [-2.62502038e+00],
       [-2.32359917e+00],
       [-2.79931338e+00],
       [-3.90553689e+00],
       [-3.31822493e+00],
       [-1.77446059e+00],
       [-3.50937021e+00],
       [-1.35002155e+00],
       [-3.25745020e+00],
       [-2.32798651e+00],
       [-1.50608337e+00],
       [-1.52196823e+00],
       [ 3.46130498e-01],
       [-2.82005824e+00],
       [ 9.99395348e-01],
       [-7.00706712e-01],
       [-1.00448556e+00],
       [ 8.79331104e-01],
       [-3.30613813e+00],
       [-1.24899821e+00],
       [ 2.72290455e-01],
       [ 1.07815161e-01],
       [ 9.30876057e-01],
       [-2.37005505e+00],
       [-1.37086248e-01],
       [-8.28772370e-01],
       [ 1.2

In [9]:
output = np.load(srcs + 'output.npy')
output

array([-1.83044863e+00, -5.41467011e-01, -3.00664306e-01,  1.31901681e+00,
        6.21483386e-01,  1.72802401e+00,  3.11243355e-01, -7.72666037e-01,
       -2.33660650e+00, -4.15072441e+00, -3.64292860e+00, -5.22179842e+00,
       -2.62502003e+00, -2.32359838e+00, -2.79931402e+00, -3.90553617e+00,
       -3.31822491e+00, -1.77446210e+00, -3.50937057e+00, -1.35002208e+00,
       -3.25744963e+00, -2.32798791e+00, -1.50608385e+00, -1.52196944e+00,
        3.46129864e-01, -2.82005906e+00,  9.99394178e-01, -7.00707018e-01,
       -1.00448704e+00,  8.79330397e-01, -3.30613828e+00, -1.24899888e+00,
        2.72289366e-01,  1.07814811e-01,  9.30875421e-01, -2.37005591e+00,
       -1.37086853e-01, -8.28773081e-01,  1.24618733e+00, -2.06461176e-01,
        8.70270550e-01,  3.38236809e-01, -2.27908686e-01,  3.72612166e+00,
        5.17988384e-01, -1.11134982e+00, -1.79146215e-01,  1.40858257e+00,
       -1.49168777e+00, -2.34599805e+00,  3.79817225e-02,  4.23640013e+00,
        7.20686093e-02, -